# Geolocation

In [1]:
import csv
import os
import re
from collections import defaultdict
from collections import Counter
from datetime import datetime
from datetime import timedelta
from string import punctuation

import matplotlib
import numpy as np
import networkx as nx
import pandas as pd
import requests
import seaborn as sns
from geolocation.main import GoogleMaps
from matplotlib import pyplot as plt
from SPARQLWrapper import SPARQLWrapper, JSON

from googlemaps_key import KEY

# Autores

In [2]:
%%time

dtypes = {
    'TITULO': str,
    'AUTOR': str,
    'PAIS_EJEMPLAR': str,
    'PAIS_AUTOR': str,
    'CAPITAL_EJEMPLAR': str,
    'CAPITAL_AUTOR': str,
}

autores_df = pd.read_csv('autores6.csv', header=0, dtype=dtypes)
autores_df = autores_df.fillna('')

CPU times: user 171 ms, sys: 19.9 ms, total: 191 ms
Wall time: 190 ms


In [3]:
autores_df.shape

(76722, 6)

In [4]:
autores_df.head()

,TITULO,PAIS_EJEMPLAR,AUTOR,PAIS_AUTOR,CAPITAL_AUTOR,CAPITAL_EJEMPLAR
0,Adivina en qué pais ...,Spain,Martina Badstuber,Germany,Berlin,Madrid
1,¡No quiero hacer pipí en el orinal!,Spain,Roser Rius,Mexico,Mexico City,Madrid
2,Humo,Spain,Antón Fortes Torres,Spain,Madrid,Madrid
3,Galleta para perros,Spain,Helen Cooper,United Kingdom,London,Madrid
4,Autobio,Spain,Cyril Pedrosa,France,Paris,Madrid


# Geocoordinates

In [5]:
autores_df['CAPITAL_EJEMPLAR'].nunique()

117

In [6]:
autores_df['CAPITAL_AUTOR'].nunique()

204

In [7]:
places = set(autores_df['CAPITAL_AUTOR'].unique()).union(autores_df['CAPITAL_EJEMPLAR'].unique())

In [8]:
len(places)

249

In [9]:
google_maps = GoogleMaps(api_key=KEY)

In [10]:
%%time

locs = {} # (lat, lng)

for x in places:
    try:
        locations = google_maps.search(location=x)
        if len(locations.list_data) > 1:
            print('MORE THAN ONE LOCATION:', x)
        location = locations.first()
        locs[x] = (location.lat, location.lng)
    except:
        print('NOT LOCATED:', x)

MORE THAN ONE LOCATION: DESCONOCIDO
NOT LOCATED: Kingdom of England
MORE THAN ONE LOCATION: Kingston
NOT LOCATED: Monaco
MORE THAN ONE LOCATION: Lima
NOT LOCATED: St. John's
NOT LOCATED: Batavia, Dutch East Indies
NOT LOCATED: Hong Kong
NOT LOCATED: Tagsatzung
NOT LOCATED: Gibraltar
MORE THAN ONE LOCATION: Susa
NOT LOCATED: Georgetown
NOT LOCATED: Singapore
MORE THAN ONE LOCATION: Brunswick
NOT LOCATED: Tokyo
NOT LOCATED: Jakarta
NOT LOCATED: Brasília
NOT LOCATED: Vatican City
CPU times: user 5.99 s, sys: 191 ms, total: 6.18 s
Wall time: 2min 52s


In [20]:
x='Brazilia'
locations = google_maps.search(location=x)
locations.list_data

[<LocationModel: b'Bras\xc3\xadlia'>]

In [22]:
f=locations.first()

In [24]:
f.city

b'Bras\xc3\xadlia'

In [11]:
%%time

def process_geolocation_capital_autor(x):
    c = x['CAPITAL_AUTOR']
    return locs.get(c, (0, 0))

autores_df['GEOLOC_CAPITAL_AUTOR'] = autores_df.apply(process_geolocation_capital_autor, axis=1)

CPU times: user 2.07 s, sys: 11 µs, total: 2.07 s
Wall time: 2.07 s


In [12]:
%%time

def process_geolocation_capital_ejemplar(x):
    c = x['CAPITAL_EJEMPLAR']
    return locs.get(c, (0, 0))

autores_df['GEOLOC_CAPITAL_EJEMPLAR'] = autores_df.apply(process_geolocation_capital_ejemplar, axis=1)

CPU times: user 1.62 s, sys: 2.94 ms, total: 1.62 s
Wall time: 1.62 s


# Saving results

In [13]:
autores_df.to_csv('autores7.csv', index=False)